In [206]:
import csv
import numpy as np
from random import shuffle, sample, choice
from math import floor

In [341]:
fname = 'kittens.in'

In [342]:
ans = open(fname).read().splitlines()

## Parameters:

In [343]:
video_num, endpoint_num, request_num, cache_num, cache_size_limit = map(int, ans[0].split())

In [344]:
video_sizes = list(enumerate(map(int, ans[1].split())))

## Endpoints

In [345]:
rest = ans[2:]
endpoints = []
point = 0
for endpoint in range(endpoint_num):
    end_latent, num_connected = map(int,rest[point].split())
    caches = []
    for l in range(num_connected):
        caches.append(map(int,rest[point+l+1].split()))
    endpoints.append([end_latent, num_connected, caches])
    point += 1 + num_connected

## Requests

In [346]:
rest = rest[point:]

requests = []
for l in range(request_num):
    requests.append(map(int, rest[l].split()))

In [319]:
def fill_server(capacity, videos):
    curr = 0
    ret = set()
    indices = list(enumerate(videos))
    shuffle(indices)
    for i, size in indices:
        if curr + size > capacity:
            continue
        ret.add(i)
        curr += size
    return ret

In [335]:
def initial_guess():
    guess = []
    for c in range(cache_num):
#         print c
        sh = sample(video_sizes, len(video_sizes))
        ans = []
        size = cache_size_limit
        while len(sh) > 0:
            if sh[0][1] < size:
                ans.append(sh[0][0])
                size -= sh[0][1]
            del sh[0]
        guess.append([c,ans])
    return guess

In [321]:
def mutate(instance):
    cacheToReplace = choice(instance)
    videoToReplace = choice(cacheToReplace[1])
    for l in range(mutateRetry):
        newVid = choice(video_sizes)[1]
        if newVid not in cacheToReplace and newVid != videoToReplace:
            cacheToReplace[1].remove(videoToReplace)
            cacheToReplace[1].append(newVid)
            if sum([l[1] for l in video_sizes]) <= cache_size_limit:
                return 
            cacheToReplace[1].remove(newVid)
            cacheToReplace[1].append(videoToReplace)

In [322]:
def mate(inst1, inst2):
    cut = int(min(len(inst1), len(inst2))/2.0)
    return inst1[:cut] + inst2[cut:]

In [323]:
def fit(solution):
    saved_time = 0 # in ms
    total_watches = 0
    
    for video_id, endpoint_id, times in requests:
        endpoint = endpoints[endpoint_id]
        # dc latency
        min_latency = endpoint[0]
        # iterate caches
        for cache, latency in endpoint[2]:
            if video_id in solution[cache][1]:
                min_latency = min(min_latency, latency)
        
        saved_latency = endpoint[0] - min_latency

#         print 'adding saved time: %i * %i' % (times, saved_latency)
        saved_time += saved_latency * times
#         print 'adding times: %i' % (times, )
        total_watches += times

    # average
    a = floor(1000.0 * saved_time / total_watches)
#     print a
    return a

In [338]:
couples = 2
pop_size = couples**2
generations = 1000
mutate_prob = 0.0

In [339]:
def run_evol():
    best_fitted = [0,0]
    pop = [initial_guess() for a in range(pop_size)]
    for gen in range(generations):
#         print pop
        print "Generation ",gen
        fited = [[p,fit(p)] for i,p in enumerate(pop)]
        fited.sort(key = lambda x: x[1])
        best = fited[:20]
#         print fited[-1]

        if best[0][1] > best_fitted[1]:
            best_fitted = best[0]
            print best_fitted
        best = [l[0] for l in best]
        new_pop = []
        for a in best:
            for b in best:
                new_pop.append(mate(a,b))
        for i,a in enumerate(new_pop):
            if np.random.rand() < mutate_prob:
                mutate(new_pop[i])
                
        pop = new_pop
    
        
#     print init

In [340]:
run_evol()

Generation  0
[[[0, [9373, 3503, 8486, 9047, 3974, 7146, 5206, 8540, 2481, 178, 5644, 5837, 257, 7763, 5795, 4657, 4784, 328, 6968, 6456, 1369, 3399, 6462, 3260, 2673, 7826, 8636, 9066, 3830, 1233, 5209, 8778, 1077, 5705, 2508, 1763, 4513, 6431, 7636, 1816, 6331, 1304, 6470, 9456, 8620, 272, 13, 9433, 1570, 1064, 964, 5519, 6167, 6750, 4303, 6000, 1164, 4253, 2148, 3499, 970, 762, 7012, 1880, 3168, 4755, 4395, 3027, 3961, 3293, 2703, 1203, 2593, 537, 3374, 3767, 2842, 6653, 6058, 5272, 6275, 457, 202, 4825, 1255, 2624, 2771, 8550, 7412, 8421]], [1, [8456, 2458, 8936, 2600, 3446, 6022, 3349, 3143, 1646, 2514, 321, 1127, 471, 5374, 6751, 5577, 6372, 1676, 5042, 5591, 5546, 8213, 9631, 4689, 1062, 5414, 5747, 1686, 135, 9289, 1461, 9743, 8315, 9628, 9973, 4291, 913, 1882, 7121, 8954, 8663, 3616, 7514, 9827, 6422, 389, 2715, 1688, 4966, 4842, 1468, 1752, 4977, 3621, 7956, 2347, 249, 2809, 5728, 1366, 1509, 726, 4970, 7416, 3866, 3365, 7646, 4730, 4929, 5409, 312, 3184, 5866, 4163, 1054, 60

KeyboardInterrupt: 

In [350]:
with open('aaa','w') as f:
    for l in initial_guess():
        f.write(str(l)+'\n')